In [1]:
import os
import torch # 파이토치 패키지 임포트
import torch.nn as nn # 자주 사용하는 torch.nn패키지를 별칭 nn으로 명명
# 허깅페이스의 트랜스포머 패키지에서 BertConfig, BertModel 클래스 임포트
from transformers import BertConfig, BertModel

In [2]:
# 전처리된 데이터가 저장된 디렉터리
DB_PATH=f'../input/processed'

# 토큰을 인덱스로 치환할 때 사용될 사전 파일이 저장된 디렉터리 
VOCAB_DIR=os.path.join(DB_PATH, 'vocab')

# 학습된 모델의 파라미터가 저장될 디렉터리
MODEL_PATH=f'../model'

In [3]:
# 미리 정의된 설정 값
class CFG:
    learning_rate=3.0e-4 # 러닝 레이트
    batch_size=1024 # 배치 사이즈
    num_workers=4 # 워커의 개수
    print_freq=100 # 결과 출력 빈도
    start_epoch=0 # 시작 에폭
    num_train_epochs=10 # 학습할 에폭수
    warmup_steps=100 # lr을 서서히 증가시킬 step 수
    max_grad_norm=10 # 그래디언트 클리핑에 사용
    weight_decay=0.01
    dropout=0.2 # dropout 확률
    hidden_size=512 # 은닉 크기
    intermediate_size=256 # TRANSFORMER셀의 intermediate 크기
    nlayers=2 # BERT의 층수
    nheads=8 # BERT의 head 개수
    seq_len=64 # 토큰의 최대 길이
    n_b_cls = 57 + 1 # 대카테고리 개수
    n_m_cls = 552 + 1 # 중카테고리 개수
    n_s_cls = 3190 + 1 # 소카테고리 개수
    n_d_cls = 404 + 1 # 세카테고리 개수
    vocab_size = 32000 # 토큰의 유니크 인덱스 개수
    img_feat_size = 2048 # 이미지 피처 벡터의 크기
    type_vocab_size = 30 # 타입의 유니크 인덱스 개수
    csv_path = os.path.join(DB_PATH, 'train.csv')
    h5_path = os.path.join(DB_PATH, 'train_img_feat.h5')

In [4]:
class CateClassifier(nn.Module):
    """상품정보를 받아서 대/중/소/세 카테고리를 예측하는 모델    
    """
    def __init__(self, cfg):        
        """
        매개변수
        cfg: hidden_size, nlayers 등 설정값을 가지고 있는 변수
        """
        super(CateClassifier, self).__init__()
        # 글로벌 설정값을 멤버 변수로 저장
        self.cfg = cfg
        # 버트모델의 설정값을 멤버 변수로 저장
        self.bert_cfg = BertConfig( 
            cfg.vocab_size, # 사전 크기
            hidden_size=cfg.hidden_size, # 히든 크기
            num_hidden_layers=cfg.nlayers, # 레이어 층 수
            num_attention_heads=cfg.nheads, # 어텐션 헤드의 수
            intermediate_size=cfg.intermediate_size, # 인터미디어트 크기
            hidden_dropout_prob=cfg.dropout, # 히든 드롭아웃 확률 값
            attention_probs_dropout_prob=cfg.dropout, # 어텐션 드롭아웃 확률 값 
            max_position_embeddings=cfg.seq_len, # 포지션 임베딩의 최대 길이
            type_vocab_size=cfg.type_vocab_size, # 타입 사전 크기
        )
        # 텍스트 인코더로 버트모델 사용
        self.text_encoder = BertModel(self.bert_cfg)
        # 이미지 인코더로 선형모델 사용(대회에서 이미지가 아닌 img_feat를 제공)
        self.img_encoder = nn.Linear(cfg.img_feat_size, cfg.hidden_size)
                
        # 분류기(Classifier) 생성기
        def get_cls(target_size=1):
            return nn.Sequential(
                nn.Linear(cfg.hidden_size*2, cfg.hidden_size),
                nn.LayerNorm(cfg.hidden_size),
                nn.Dropout(cfg.dropout),
                nn.ReLU(),
                nn.Linear(cfg.hidden_size, target_size),
            )        
          
        # 대 카테고리 분류기
        self.b_cls = get_cls(cfg.n_b_cls)
        # 중 카테고리 분류기
        self.m_cls = get_cls(cfg.n_m_cls)
        # 소 카테고리 분류기
        self.s_cls = get_cls(cfg.n_s_cls)
        # 세 카테고리 분류기
        self.d_cls = get_cls(cfg.n_d_cls)
    
    def forward(self, token_ids, token_mask, token_types, img_feat, label=None):
        """        
        매개변수
        token_ids: 전처리된 상품명을 인덱스로 변환하여 token_ids를 만들었음
        token_mask: 실제 token_ids의 개수만큼은 1, 나머지는 0으로 채움
        token_types: ▁ 문자를 기준으로 서로 다른 타입의 토큰임을 타입 인덱스로 저장
        img_feat: resnet50으로 인코딩된 이미지 피처
        label: 정답 대/중/소/세 카테고리
        """

        # 전처리된 상품명을 하나의 텍스트벡터(text_vec)로 변환
        # 반환 튜플(시퀀스 아웃풋, 풀드(pooled) 아웃풋) 중 시퀀스 아웃풋만 사용
        text_output = self.text_encoder(token_ids, token_mask, token_type_ids=token_types)[0] # (batch_size, sequence_length, hidden_size)
         
        # 시퀀스 중 첫 타임스탭의 hidden state만 사용. 
        text_vec = text_output[:, 0] # (batch_size, text_hidden_size)
        
        # img_feat를 텍스트벡터와 결합하기 앞서 선형변환 적용
        img_vec = self.img_encoder(img_feat) #(batch_size, img_hidden_size)
        
        # 이미지벡터와 텍스트벡터를 직렬연결(concatenate)하여 결합벡터 생성
        comb_vec = torch.cat([text_vec, img_vec], 1) # (batch_size, hidden_size*2)
        
        # 결합된 벡터로 대카테고리 확률분포 예측
        b_pred = self.b_cls(comb_vec)
        # 결합된 벡터로 중카테고리 확률분포 예측
        m_pred = self.m_cls(comb_vec)
        # 결합된 벡터로 소카테고리 확률분포 예측
        s_pred = self.s_cls(comb_vec)
        # 결합된 벡터로 세카테고리 확률분포 예측
        d_pred = self.d_cls(comb_vec)
        
        # 데이터 패러럴 학습에서 GPU 메모리를 효율적으로 사용하기 위해 
        # loss를 모델 내에서 계산함.
        if label is not None:
            # 손실(loss) 함수로 CrossEntropyLoss를 사용
            # label의 값이 -1을 가지는 샘플은 loss계산에 사용 안 함
            loss_func = nn.CrossEntropyLoss(ignore_index=-1)
            # label은 batch_size x 4를 (batch_size x 1) 4개로 만듦
            b_label, m_label, s_label, d_label = label.split(1, 1)
            # 대카테고리의 예측된 확률분포와 정답확률 분포의 차이를 손실로 반환
            b_loss = loss_func(b_pred, b_label.view(-1))
            # 중카테고리의 예측된 확률분포와 정답확률 분포의 차이를 손실로 반환
            m_loss = loss_func(m_pred, m_label.view(-1))
            # 소카테고리의 예측된 확률분포와 정답확률 분포의 차이를 손실로 반환
            s_loss = loss_func(s_pred, s_label.view(-1))
            # 세카테고리의 예측된 확률분포와 정답확률 분포의 차이를 손실로 반환
            d_loss = loss_func(d_pred, d_label.view(-1))
            # 대/중/소/세 손실의 평균을 낼 때 실제 대회 평가방법과 일치하도록 함
            loss = (b_loss + 1.2*m_loss + 1.3*s_loss + 1.4*d_loss)/4    
        else: # label이 없으면 loss로 0을 반환
            loss = b_pred.new(1).fill_(0)      
        
        # 최종 계산된 손실과 예측된 대/중/소/세 각 확률분포를 반환
        return loss, [b_pred, m_pred, s_pred, d_pred]

In [24]:
cfg = CFG()

model = CateClassifier(cfg)

In [25]:
def count_parameters(model):
        return sum(p.numel() for p in model.parameters() if p.requires_grad)

print('parameters: ', count_parameters(model))

parameters:  24637551


In [26]:
param_optimizer = list(model.named_parameters())    
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params':[p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [32]:
for n,p in param_optimizer:
        if any(nd in n for nd in no_decay):
            print(n)

text_encoder.embeddings.LayerNorm.weight
text_encoder.embeddings.LayerNorm.bias
text_encoder.encoder.layer.0.attention.self.query.bias
text_encoder.encoder.layer.0.attention.self.key.bias
text_encoder.encoder.layer.0.attention.self.value.bias
text_encoder.encoder.layer.0.attention.output.dense.bias
text_encoder.encoder.layer.0.attention.output.LayerNorm.weight
text_encoder.encoder.layer.0.attention.output.LayerNorm.bias
text_encoder.encoder.layer.0.intermediate.dense.bias
text_encoder.encoder.layer.0.output.dense.bias
text_encoder.encoder.layer.0.output.LayerNorm.weight
text_encoder.encoder.layer.0.output.LayerNorm.bias
text_encoder.encoder.layer.1.attention.self.query.bias
text_encoder.encoder.layer.1.attention.self.key.bias
text_encoder.encoder.layer.1.attention.self.value.bias
text_encoder.encoder.layer.1.attention.output.dense.bias
text_encoder.encoder.layer.1.attention.output.LayerNorm.weight
text_encoder.encoder.layer.1.attention.output.LayerNorm.bias
text_encoder.encoder.layer.1

In [15]:
a = torch.rand(32, 10)
b = torch.rand(32, 20)
c = torch.rand(32, 30)
d = [a,b,c]

pred_list = []
pred_list.append(d)

In [17]:
pred_list.append(d)

In [18]:
len(pred_list)

2

In [20]:
pred_list[0][0].shape

torch.Size([32, 10])

In [1]:
import pandas as pd

df = pd.read_csv('/home/yyeon/KeepGo/My-Competition-Struggle/Kakao_Arena/categories-prediction/input/processed/train.csv')

In [2]:
df.head()

,pid,tokens,bcateid,mcateid,scateid,dcateid
0,O4486751463,▁직소퍼즐 ▁1000 조각 ▁바다 거북 의 ▁여행 ▁pl 12 75,1,1,2,-1
1,P3307178849,▁모리케이스 ▁아이폰 6 s ▁6 s ▁tree ▁farm 101 ▁다이어리케이스 ...,3,3,4,-1
2,R4424255515,▁크리비아 ▁기모 ▁3 부 ▁속바지 ▁gl g 43 14 p,5,5,6,-1
3,F3334315393,▁하프클럽 ▁잭앤질 ▁남성 ▁솔리드 ▁절개라인 ▁포인트 ▁포켓 ▁팬츠 ▁311 33...,7,7,8,-1
4,N731678492,▁코드 프리 혈 당 시험 지 50 매 ▁코드 프리 시험 지 ▁최 장 유 효 기간,10,9,11,-1


In [6]:
df['unique_cateid'] = (df['bcateid'].astype('str') +
                       df['mcateid'].astype('str') + 
                       df['scateid'].astype('str') + df['dcateid'].astype('str')).astype('category')
df['unique_cateid'] = df['unique_cateid'].cat.codes

In [8]:
df.head()

,pid,tokens,bcateid,mcateid,scateid,dcateid,unique_cateid
0,O4486751463,▁직소퍼즐 ▁1000 조각 ▁바다 거북 의 ▁여행 ▁pl 12 75,1,1,2,-1,109
1,P3307178849,▁모리케이스 ▁아이폰 6 s ▁6 s ▁tree ▁farm 101 ▁다이어리케이스 ...,3,3,4,-1,2295
2,R4424255515,▁크리비아 ▁기모 ▁3 부 ▁속바지 ▁gl g 43 14 p,5,5,6,-1,3720
3,F3334315393,▁하프클럽 ▁잭앤질 ▁남성 ▁솔리드 ▁절개라인 ▁포인트 ▁포켓 ▁팬츠 ▁311 33...,7,7,8,-1,3944
4,N731678492,▁코드 프리 혈 당 시험 지 50 매 ▁코드 프리 시험 지 ▁최 장 유 효 기간,10,9,11,-1,49


In [10]:
df['unique_cateid'].nunique()

4214

In [15]:
from sklearn.model_selection import StratifiedKFold, KFold

folds = KFold(n_splits =5, shuffle=True)

train_idx, valid_idx = list(folds.split(df.values))[0]

In [18]:
len(train_idx) / df.shape[0]

0.7999999508286479

In [5]:
import torch # 파이토치 패키지 임포트
from torch.utils.data import Dataset # Dataset 클래스 임포트
import h5py # h5py 패키지 임포트
import re # 정규식표현식 모듈 임포트 

class CateDataset(Dataset):
    """데이터셋에서 학습에 필요한 형태로 변환된 샘플 하나를 반환
    """
    def __init__(self, df_data, img_h5_path, token2id, tokens_max_len=64, type_vocab_size=30):
        """        
        매개변수
        df_data: 상품타이틀, 카테고리 등의 정보를 가지는 데이터프레임
        img_h5_path: img_feat가 저장돼 있는 h5 파일의 경로
        token2id: token을 token_id로 변환하기 위한 맵핑 정보를 가진 딕셔너리
        tokens_max_len: tokens의 최대 길이. 상품명의 tokens가 이 이상이면 잘라서 버림
        type_vocab_size: 타입 사전의 크기
        """        
        self.tokens = df_data['tokens'].values # 전처리된 상품명
        self.img_indices = df_data['img_idx'].values # h5의 이미지 인덱스
        self.img_h5_path = img_h5_path 
        self.tokens_max_len = tokens_max_len        
        self.labels = df_data[['bcateid', 'mcateid', 'scateid', 'dcateid']].values
        self.token2id = token2id 
        self.p = re.compile('▁[^▁]+') # ▁기호를 기준으로 나누기 위한 컴파일된 정규식
        self.type_vocab_size = type_vocab_size
        
    def __getitem__(self, idx):
        """
        데이터셋에서 idx에 대응되는 샘플을 변환하여 반환        
        """
        if idx >= len(self):
            raise StopIteration
        
        # idx에 해당하는 상품명 가져오기. 상품명은 문자열로 저장돼 있음
        tokens = self.tokens[idx]
        if not isinstance(tokens, str):
            tokens = ''
        
        # 상품명을 ▁기호를 기준으로 분리하여 파이썬 리스트로 저장
        # "▁직소퍼즐 ▁1000 조각 ▁바다 거북 의 ▁여행 ▁pl 12 75" =>
        # ["▁직소퍼즐", "▁1000 조각", "▁바다 거북 의", "▁여행", "▁pl 12 75"]
        tokens = self.p.findall(tokens)
        
        # ▁ 기호 별 토큰타입 인덱스 부여
        # ["▁직소퍼즐", "▁1000 조각", "▁바다 거북 의", "▁여행", "▁pl 12 75"] =>
        # [     0     ,     1    1  ,    2     2  2 ,     3   ,   4  4   4 ]
        token_types = [type_id for type_id, word in enumerate(tokens) for _ in word.split()]       
        tokens = " ".join(tokens) # ▁기호로 분리되기 전의 원래의 tokens으로 되돌림

        # 토큰을 토큰에 대응되는 인덱스로 변환
        # "▁직소퍼즐 ▁1000 조각 ▁바다 거북 의 ▁여행 ▁pl 12 75" =>
        # [2291, 784, 2179, 3540, 17334, 30827, 1114, 282, 163, 444]
        # "▁직소퍼즐" => 2291
        # "▁1000" => 784
        # "조각" => 2179
        # ...
        token_ids = [self.token2id[tok] if tok in self.token2id else 0 for tok in tokens.split()]
        
        # token_ids의 길이가 max_len보다 길면 잘라서 버림
        if len(token_ids) > self.tokens_max_len:
            token_ids = token_ids[:self.tokens_max_len]      
            token_types = token_types[:self.tokens_max_len]
        
        # token_ids의 길이가 max_len보다 짧으면 짧은만큼 PAD값 0 값으로 채워넣음
        # token_ids 중 값이 있는 곳은 1, 그 외는 0으로 채운 token_mask 생성
        token_mask = [1] * len(token_ids)
        token_pad = [0] * (self.tokens_max_len - len(token_ids))
        token_ids += token_pad
        token_mask += token_pad
        token_types += token_pad # max_len 보다 짧은만큼 PAD 추가

        # h5파일에서 이미지 인덱스에 해당하는 img_feat를 가져옴
        # 파이토치의 데이터로더에 의해 동시 h5파일에 동시접근이 발생해도
        # 안정적으로 img_feat를 가져오려면 아래처럼 매번 h5py.File 호출필요
        with h5py.File(self.img_h5_path, 'r') as img_feats:
            img_feat = img_feats['img_feat'][self.img_indices[idx]]
        
        # 넘파이(numpy)나 파이썬 자료형을 파이토치의 자료형으로 변환
        token_ids = torch.LongTensor(token_ids)
        token_mask = torch.LongTensor(token_mask)
        token_types = torch.LongTensor(token_types)
        
        # token_types의 타입 인덱스의 숫자 크기가 type_vocab_size 보다 작도록 바꿈
        token_types[token_types >= self.type_vocab_size] = self.type_vocab_size-1 
        img_feat = torch.FloatTensor(img_feat)
        
        # 대/중/소/세 라벨 준비
        label = self.labels[idx]
        label = torch.LongTensor(label)
        
        # 크게 3가지 텍스트 입력, 이미지 입력, 라벨을 반환한다.
        return token_ids, token_mask, token_types, img_feat, label
    
    def __len__(self):
        """
          tokens의 개수를 반환한다. 즉, 상품명 문장의 개수를 반환한다.
        """
        return len(self.tokens)


In [3]:
tokens = ["▁직소퍼즐", "▁1000 조각", "▁바다 거북 의", "▁여행", "▁pl 12 75"]

In [4]:
token_types = [type_id for type_id, word in enumerate(tokens) for _ in word.split()]

In [5]:
token_types

[0, 1, 1, 2, 2, 2, 3, 4, 4, 4]

In [8]:
import glob

In [12]:
model_list = []
# args.model_dir에 있는 확장자 .pt를 가지는 모든 모델 파일의 경로를 읽음
model_path_list = glob.glob(os.path.join(MODEL_PATH, '*.pt'))
# 모델 경로 개수만큼 모델을 생성하여 파이썬 리스트에 추가함
for model_path in model_path_list:
    model = CateClassifier(CFG)
    if model_path != "":
        print("=> loading checkpoint '{}'".format(model_path))
        checkpoint = torch.load(model_path)        
        state_dict = checkpoint['state_dict']                
        model.load_state_dict(state_dict, strict=True)  
        print("=> loaded checkpoint '{}' (epoch {})".format(model_path, checkpoint['epoch']))
    model.cuda()
    n_gpu = torch.cuda.device_count()
    if n_gpu > 1:
        model = torch.nn.DataParallel(model)
    model_list.append(model)
    
if len(model_list) == 0:
    print('Please check the model directory.')

=> loading checkpoint '../model/b1024_h512_d0.2_l2_hd8_ep2_s7_fold0.pt'
=> loaded checkpoint '../model/b1024_h512_d0.2_l2_hd8_ep2_s7_fold0.pt' (epoch 3)


In [14]:
model_list

[CateClassifier(
   (text_encoder): BertModel(
     (embeddings): BertEmbeddings(
       (word_embeddings): Embedding(32000, 512, padding_idx=0)
       (position_embeddings): Embedding(64, 512)
       (token_type_embeddings): Embedding(30, 512)
       (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
       (dropout): Dropout(p=0.2, inplace=False)
     )
     (encoder): BertEncoder(
       (layer): ModuleList(
         (0): BertLayer(
           (attention): BertAttention(
             (self): BertSelfAttention(
               (query): Linear(in_features=512, out_features=512, bias=True)
               (key): Linear(in_features=512, out_features=512, bias=True)
               (value): Linear(in_features=512, out_features=512, bias=True)
               (dropout): Dropout(p=0.2, inplace=False)
             )
             (output): BertSelfOutput(
               (dense): Linear(in_features=512, out_features=512, bias=True)
               (LayerNorm): LayerNorm((512,), eps